<a href="https://colab.research.google.com/github/JacobItopa/cluster_analysis/blob/main/clusterring_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering

## Key Components of Cluster Analysis

* Distance/similarity/dissimilarity function
* Loss function to evaluate clusters
* Algorithm to optimize loss function
* Goal:  Find natural groupings in data
  * Form of unsupervised learning
  * Often the "correct" groupings are unknown
  * Samples within a group should be more similar than samples in different groups

## Distance Metrics

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/2880px-Euclidean_distance_2d.svg.png" width="400">

Distance metrics (Hamming, Euclidean, Cosine, Mahalanobis)